# Preprocessing script for ECoG Data -- written by Yvonne Fonken Dec 2016


In [1]:
#Import packages
import numpy as np
import pandas as pd
import preprocessYF as pre
import os
import mne
import os, glob, sys

/usr/local/anaconda-2.4.0/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/anaconda-2.4.0/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Get all blocknames
# the raw fif files we're going to use
sub_dir = '/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/'
file_str = sub_dir + 'B*/AB62raw_Dig.fif' # Can include wildcards, e.g. 'B0*raw.fif'
files = glob.glob(file_str) # All files matching pattern
print '\n'.join(files)

/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B03/AB62raw_Dig.fif
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B04/AB62raw_Dig.fif
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B05/AB62raw_Dig.fif
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B06/AB62raw_Dig.fif
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B07/AB62raw_Dig.fif
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B08/AB62raw_Dig.fif


In [3]:
# get the event files also
evfile_str = file_str[:-4] + '-eve.fif' # names violate MNE conventions, need to end with raw.fif
evfiles = glob.glob(evfile_str)
print '\n'.join(evfiles)

/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B03/AB62raw_Dig-eve.fif
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B04/AB62raw_Dig-eve.fif
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B05/AB62raw_Dig-eve.fif
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B06/AB62raw_Dig-eve.fif
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B07/AB62raw_Dig-eve.fif
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B08/AB62raw_Dig-eve.fif


In [4]:
for ixf in range(len(files)): # added 1: here to skip practice block
    f = files[ixf]
    f_evs = evfiles[ixf]
    resample = 12
    
    # Load the data from fif file, preprocess, reference
    ev_id = dict(la=11, ba=12, ga=13, ta=15, expected_ba=22, expected_ga=23) 
    # Or
    #Control for AB71, AB81
    #ev_id = dict(omission_control = 12, other1 = 11, other2 = 13, other3 = 15, other4 = 16)
    #ev_id = dict(la=11, ba=12, ga=13, ta=15, la_control = 16, ta_control = 17, omission_control = 18)
    # Or control and main experiment combined: 
    #ev_id = dict( la=11, ba=12, ga=13, ta=15, la_control = 16, ta_control = 17, omission_control = 18, expected_ba = 22, expected_ga = 23)
    
    raw_array = pre.load_block(f, ev_id=ev_id, bp_freq = (1.0, 190.0), t_discard = (10.0, 5.0), ref_method = 'car', fcar = 'CarGroup.csv')
    
    # Epoch data, save ERP data

    #epochsERP = pre.make_epochs(f_evs = f_evs, ev_id= ev_id, raw_array = raw_array, t = (-1.5, 1.5), resample = resample, f_output_name = 'ERP_DS_Dig10Hz')


    # Do Hilbert transform
    Flow = 13
    Fhigh = 30
    raw_array = pre.hg_filter(raw_array, Flow, Fhigh)
    # Save epoched hg data
    epochsHG = pre.make_epochs(f_evs = f_evs, ev_id = ev_id, raw_array = raw_array, resample = resample, t= (-1.5, 1.5), f_output_name = 'HilbertBeta_Dig')
    
    Flow = 30
    Fhigh = 55
    raw_array = pre.hg_filter(raw_array, Flow, Fhigh)
    # Save epoched hg data
    epochsHG = pre.make_epochs(f_evs = f_evs, ev_id = ev_id, raw_array = raw_array, resample = resample, t= (-1.5, 1.5), f_output_name = 'HilbertGamma_Dig')
    

Files:
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B03/AB62raw_Dig.fif
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B03/BadChannels.csv
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B03/CarGroup.csv

Bad channels are: [7, 8, 9, 10, 12, 17, 18, 19, 20, 27, 45, 46, 48, 49, 50, 80, 99, 121, 135, 136, 137, 138, 185, 186, 209]
Opening raw data file /home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B03/AB62raw_Dig.fif...
This filename (/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B03/AB62raw_Dig.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
Isotrak not found
    Range : 0 ... 369599 =      0.000 ...   307.999 secs
Ready.
Reading 0 ... 369599  =      0.000 ...   307.999 secs...
There are 369600 timepoints.
Creating RawArray with float64 data, n_channels=256, n_times=369600
    Range : 0 ... 369599 =      0.000 

 'ba': 65, 'expected_ba': 10, 'expected_ga': 9, 'ga': 65, 'la': 313, 'ta': 8>
(470, 3)
now saving to...
Data
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62
epochs right before saving
<Epochs  |  n_events : 470 (good & bad), tmin : -1.5 (s), tmax : 1.5 (s), baseline : (None, 0), ~466 kB, data not loaded,
 'ba': 65, 'expected_ba': 10, 'expected_ga': 9, 'ga': 65, 'la': 313, 'ta': 8>
<Info | 16 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : list | EEG 1, EEG 2, EEG 3, EEG 4, EEG 5, EEG 6, EEG 11, ...
    chs : list | 229 items (MISC: 229)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : 'mne.transforms.Transform | 3 items
    events : list | 0 items
    highpass : float | 13.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 25.0 Hz
    meas_date : numpy.ndarray | 1969-12-31 16:00:00
    nchan : int | 2

>
Loading data for 470 events and 3601 original time points ...
0 bad epochs dropped
data saved
Files:
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B04/AB62raw_Dig.fif
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B04/BadChannels.csv
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B04/CarGroup.csv

Bad channels are: [7, 8, 9, 10, 12, 17, 18, 19, 20, 27, 45, 46, 48, 49, 50, 80, 99, 121, 135, 136, 137, 138, 185, 186, 209]
Opening raw data file /home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B04/AB62raw_Dig.fif...
This filename (/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B04/AB62raw_Dig.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
Isotrak not found
    Range : 0 ... 369599 =      0.000 ...   307.999 secs
Ready.
Reading 0 ... 369599  =      0.000 ...   307.999 secs...
There are 369600 timepoints.
Creating R

>
Loading data for 480 events and 3601 original time points ...
0 bad epochs dropped
data saved
Setting up band-pass filter from 30 - 55 Hz
l_trans_bandwidth chosen to be 7.5 Hz
h_trans_bandwidth chosen to be 13.8 Hz
Filter length of 1056 samples (0.880 sec) selected
Applying hilbert transform...
<RawArray  |  None, n_channels x n_times : 228 x 369600 (308.0 sec), ~643.4 MB, data loaded> <Info | 16 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : list | EEG 1, EEG 2, EEG 3, EEG 4, EEG 5, EEG 6, EEG 11, ...
    chs : list | 228 items (MISC: 228)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : 'mne.transforms.Transform | 3 items
    events : list | 0 items
    highpass : float | 30.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 25.0 Hz
    meas_date : numpy.ndarray | 1969-12-31 16:00:00
    nchan : int | 228
    projs : list | 0 items
    sfreq : float | 1200.0 Hz
    a

[u'EEG 7', u'EEG 8', u'EEG 9', u'EEG 10', u'EEG 12', u'EEG 17', u'EEG 18', u'EEG 19', u'EEG 20', u'EEG 27', u'EEG 45', u'EEG 46', u'EEG 48', u'EEG 49', u'EEG 50', u'EEG 80', u'EEG 99', u'EEG 121', u'EEG 135', u'EEG 136', u'EEG 137', u'EEG 138', u'EEG 185', u'EEG 186', u'EEG 209']
-14.23046875 65234.386326
-14.23046875 2.07101264682
Setting up band-pass filter from 1 - 1.9e+02 Hz
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 47.5 Hz
Filter length of 7920 samples (6.600 sec) selected
Setting up band-stop filter
Filter length of 15840 samples (13.200 sec) selected
# Bad channels:  2 [u'EEG 182', u'EEG 238']
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B05/CarGroup.csv
Number of groups found: 1

Preprocessed data with parameters:         
Band pass frequency:  (1.0, 190.0) 
Discard times:  (10.0, 5.0) 
Reference method:  car 
White matter ref. file:  None 
Bipolar ref. direction:  out
Setting up band-pass filter from 13 - 30 Hz
l_trans_bandwidth chosen

...

> 0
Setting up low-pass filter at 25 Hz
h_trans_bandwidth chosen to be 6.2 Hz
Filter length of 1267 samples (1.056 sec) selected
<RawArray  |  None, n_channels x n_times : 226 x 369600 (308.0 sec), ~637.7 MB, data loaded> <Info | 16 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : list | EEG 1, EEG 2, EEG 3, EEG 4, EEG 5, EEG 6, EEG 13, ...
    chs : list | 226 items (MISC: 226)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : 'mne.transforms.Transform | 3 items
    events : list | 0 items
    highpass : float | 13.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 25.0 Hz
    meas_date : numpy.ndarray | 1969-12-31 16:00:00
    nchan : int | 226
    projs : list | 0 items
    sfreq : float | 1200.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    dig : NoneType
    experimenter : NoneType
 

> 0
Setting up low-pass filter at 25 Hz
h_trans_bandwidth chosen to be 6.2 Hz
Filter length of 1267 samples (1.056 sec) selected
<RawArray  |  None, n_channels x n_times : 226 x 369600 (308.0 sec), ~637.7 MB, data loaded> <Info | 16 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : list | EEG 1, EEG 2, EEG 3, EEG 4, EEG 5, EEG 6, EEG 13, ...
    chs : list | 226 items (MISC: 226)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : 'mne.transforms.Transform | 3 items
    events : list | 0 items
    highpass : float | 30.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 25.0 Hz
    meas_date : numpy.ndarray | 1969-12-31 16:00:00
    nchan : int | 226
    projs : list | 0 items
    sfreq : float | 1200.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    dig : NoneType
    experimenter : NoneType
 

[u'EEG 7', u'EEG 8', u'EEG 9', u'EEG 10', u'EEG 12', u'EEG 17', u'EEG 18', u'EEG 19', u'EEG 20', u'EEG 27', u'EEG 45', u'EEG 46', u'EEG 48', u'EEG 49', u'EEG 50', u'EEG 80', u'EEG 99', u'EEG 121', u'EEG 135', u'EEG 136', u'EEG 137', u'EEG 138', u'EEG 185', u'EEG 186', u'EEG 209']
-3.125 62645.7341135
-3.125 18.2314903618
Setting up band-pass filter from 1 - 1.9e+02 Hz
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 47.5 Hz
Filter length of 7920 samples (6.600 sec) selected
Setting up band-stop filter
Filter length of 15840 samples (13.200 sec) selected
# Bad channels:  5 [u'EEG 11', u'EEG 101', u'EEG 113', u'EEG 120', u'EEG 224']
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62/B08/CarGroup.csv
Number of groups found: 1

Preprocessed data with parameters:         
Band pass frequency:  (1.0, 190.0) 
Discard times:  (10.0, 5.0) 
Reference method:  car 
White matter ref. file:  None 
Bipolar ref. direction:  out
Setting up band-pass filter from 13 - 30 Hz


/home/knight/ECOGprediction/OmissionDetection/Data/Albany62
epochs right before saving
<Epochs  |  n_events : 461 (good & bad), tmin : -1.5 (s), tmax : 1.5 (s), baseline : (None, 0), ~460 kB, data not loaded,
 'ba': 63, 'expected_ba': 9, 'expected_ga': 10, 'ga': 63, 'la': 308, 'ta': 8>
<Info | 16 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : list | EEG 1, EEG 2, EEG 3, EEG 4, EEG 5, EEG 6, EEG 13, ...
    chs : list | 226 items (MISC: 226)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : 'mne.transforms.Transform | 3 items
    events : list | 0 items
    highpass : float | 13.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 25.0 Hz
    meas_date : numpy.ndarray | 1969-12-31 16:00:00
    nchan : int | 226
    projs : list | 0 items
    sfreq : float | 100.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_c

 'ba': 63, 'expected_ba': 9, 'expected_ga': 10, 'ga': 63, 'la': 308, 'ta': 8>
size of data in epochs after remove bad epochs
<Epochs  |  n_events : 461 (good & bad), tmin : -1.5 (s), tmax : 1.5 (s), baseline : (None, 0), ~460 kB, data not loaded,
 'ba': 63, 'expected_ba': 9, 'expected_ga': 10, 'ga': 63, 'la': 308, 'ta': 8>
(461, 3)
now saving to...
Data
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62
/home/knight/ECOGprediction/OmissionDetection/Data/Albany62
epochs right before saving
<Epochs  |  n_events : 461 (good & bad), tmin : -1.5 (s), tmax : 1.5 (s), baseline : (None, 0), ~460 kB, data not loaded,
 'ba': 63, 'expected_ba': 9, 'expected_ga': 10, 'ga': 63, 'la': 308, 'ta': 8>
<Info | 16 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : list | EEG 1, EEG 2, EEG 3, EEG 4, EEG 5, EEG 6, EEG 13, ...
    chs : list | 226 items (MISC: 226)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : 'mne.transfo

In [ ]:
f_evs



In [ ]:
#debugging
evs = mne.read_events('/home/knight/ECOGprediction/OmissionDetection/Data/CP16/CP16raw-eve.fif')

In [ ]:
evs.shape
evs

In [ ]:
indFakeTrial = evs[:,0] == 3001 
print indFakeTrial.shape
